In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle

df = pd.read_csv('insurance_claims.csv')
df = df.drop(['_c39'], axis = 1)

df['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])

drop_columns = ['policy_state', 'policy_csl', 'incident_date', 'incident_state', 'incident_city', 'incident_location']
df = df.drop(drop_columns, axis = 1)

df['fraud_reported'] = df['fraud_reported'].str.replace('Y', '1')
df['fraud_reported'] = df['fraud_reported'].str.replace('N', '0')
df['fraud_reported'] = df['fraud_reported'].astype(int)

bin_labels = ['15-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65']
bins = [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
df['age_group'] = pd.cut(df['age'], bins = bins, labels = bin_labels, include_lowest = True)

bins = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
bin_labels = ['0-50','51-100','101-150','151-200','201-250','251-300','301-350','351-400','401-450','451-500']
df['months_as_customer_groups'] = pd.cut(df['months_as_customer'], bins = 10, labels = bin_labels, include_lowest= True)

bins = list(np.linspace(0,2500, 6, dtype = int))
bin_labels = ['very low', 'low', 'medium', 'high', 'very high']
df['policy_annual_premium_groups'] = pd.cut(df['policy_annual_premium'], bins = bins, labels=bin_labels)

bins = list(np.linspace(0,2000, 5, dtype = int))
bin_labels = ['0-500', '501-1000', '1001-1500', '1501-2000']
df['policy_deductable_group'] = pd.cut(df['policy_deductable'], bins = bins, labels = bin_labels)

df = df.drop(['age', 'months_as_customer', 'policy_deductable', 'policy_annual_premium'], axis = 1)

required_columns = ['insured_sex', 'insured_occupation',
       'insured_hobbies', 'capital-gains', 'capital-loss', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'witnesses', 'total_claim_amount', 'fraud_reported', 'age_group',
       'months_as_customer_groups', 'policy_annual_premium_groups']

df1 = df[required_columns]

cat_cols = ['age_group', 'months_as_customer_groups', 'policy_annual_premium_groups']
for col in cat_cols:
  df1[col] = df1[col].astype('object')

columns_to_encode = []
for col in df1.columns:
  if df1[col].dtype == 'object':
    columns_to_encode.append(col)


df2 = pd.get_dummies(df1, columns = columns_to_encode)

features = []
for col in df2.columns:
  if col != 'fraud_reported':
    features.append(col)

target = 'fraud_reported'

X = df2[features]
y = df2[target]

C:\Users\qccs\AppData\Local\Temp\ipykernel_9336\1355387837.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col] = df1[col].astype('object')


In [29]:
X.iloc[[0]]

,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,witnesses,total_claim_amount,insured_sex_FEMALE,insured_sex_MALE,insured_occupation_adm-clerical,insured_occupation_armed-forces,...,months_as_customer_groups_301-350,months_as_customer_groups_351-400,months_as_customer_groups_401-450,months_as_customer_groups_451-500,months_as_customer_groups_51-100,policy_annual_premium_groups_high,policy_annual_premium_groups_low,policy_annual_premium_groups_medium,policy_annual_premium_groups_very high,policy_annual_premium_groups_very low
0,53300,0,5,1,2,71610,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0


In [22]:
XValArr = X.iloc[[0]]
for i in X.columns:
    for j in XValArr[i]:
        print(i + " " + str(j))

capital-gains 53300
capital-loss 0
incident_hour_of_the_day 5
number_of_vehicles_involved 1
witnesses 2
total_claim_amount 71610
insured_sex_FEMALE 0
insured_sex_MALE 1
insured_occupation_adm-clerical 0
insured_occupation_armed-forces 0
insured_occupation_craft-repair 1
insured_occupation_exec-managerial 0
insured_occupation_farming-fishing 0
insured_occupation_handlers-cleaners 0
insured_occupation_machine-op-inspct 0
insured_occupation_other-service 0
insured_occupation_priv-house-serv 0
insured_occupation_prof-specialty 0
insured_occupation_protective-serv 0
insured_occupation_sales 0
insured_occupation_tech-support 0
insured_occupation_transport-moving 0
insured_hobbies_base-jumping 0
insured_hobbies_basketball 0
insured_hobbies_board-games 0
insured_hobbies_bungie-jumping 0
insured_hobbies_camping 0
insured_hobbies_chess 0
insured_hobbies_cross-fit 0
insured_hobbies_dancing 0
insured_hobbies_exercise 0
insured_hobbies_golf 0
insured_hobbies_hiking 0
insured_hobbies_kayaking 0
insu

In [23]:
temp = {'capital-gains': 53300,
        'capital-loss': 0,
        'incident_hour_of_the_day': 5,
        'number_of_vehicles_involved': 1,
        'witnesses': 2,
        'total_claim_amount': 71610,
        'insured_sex_FEMALE': 0,
        'insured_sex_MALE': 1,
        'insured_occupation_adm-clerical': 0,
        'insured_occupation_armed-forces': 0,
        'insured_occupation_craft-repair': 1,
        'insured_occupation_exec-managerial': 0,
        'insured_occupation_farming-fishing': 0,
        'insured_occupation_handlers-cleaners': 0,
        'insured_occupation_machine-op-inspct': 0,
        'insured_occupation_other-service': 0,
        'insured_occupation_priv-house-serv': 0,
        'insured_occupation_prof-specialty': 0,
        'insured_occupation_protective-serv': 0,
        'insured_occupation_sales': 0,
        'insured_occupation_tech-support': 0,
        'insured_occupation_transport-moving': 0,
        'insured_hobbies_base-jumping': 0,
        'insured_hobbies_basketball': 0,
        'insured_hobbies_board-games': 0,
        'insured_hobbies_bungie-jumping': 0,
        'insured_hobbies_camping': 0,
        'insured_hobbies_chess': 0,
        'insured_hobbies_cross-fit': 0,
        'insured_hobbies_dancing': 0,
        'insured_hobbies_exercise': 0,
        'insured_hobbies_golf': 0,
        'insured_hobbies_hiking': 0,
        'insured_hobbies_kayaking': 0,
        'insured_hobbies_movies': 0,
        'insured_hobbies_paintball': 0,
        'insured_hobbies_polo': 0,
        'insured_hobbies_reading': 0,
        'insured_hobbies_skydiving': 0,
        'insured_hobbies_sleeping': 1,
        'insured_hobbies_video-games': 0,
        'insured_hobbies_yachting': 0,
        'incident_type_Multi-vehicle Collision': 0,
        'incident_type_Parked Car': 0,
        'incident_type_Single Vehicle Collision': 1,
        'incident_type_Vehicle Theft': 0,
        'collision_type_?': 0,
        'collision_type_Front Collision': 0,
        'collision_type_Rear Collision': 0,
        'collision_type_Side Collision': 1,
        'incident_severity_Major Damage': 1,
        'incident_severity_Minor Damage': 0,
        'incident_severity_Total Loss': 0,
        'incident_severity_Trivial Damage': 0,
        'authorities_contacted_Ambulance': 0,
        'authorities_contacted_Fire': 0,
        'authorities_contacted_None': 0,
        'authorities_contacted_Other': 0,
        'authorities_contacted_Police': 1,
        'age_group_15-20': 0,
        'age_group_21-25': 0,
        'age_group_26-30': 0,
        'age_group_31-35': 0,
        'age_group_36-40': 0,
        'age_group_41-45': 0,
        'age_group_46-50': 1,
        'age_group_51-55': 0,
        'age_group_56-60': 0,
        'age_group_61-65': 0,
        'months_as_customer_groups_0-50': 0,
        'months_as_customer_groups_51-100': 0,
        'months_as_customer_groups_101-150': 0,
        'months_as_customer_groups_151-200': 0,
        'months_as_customer_groups_201-250': 0,
        'months_as_customer_groups_251-300': 0,
        'months_as_customer_groups_301-350': 1,
        'months_as_customer_groups_351-400': 0,
        'months_as_customer_groups_401-450': 0,
        'months_as_customer_groups_451-500': 0,
        'policy_annual_premium_groups_very low': 0,
        'policy_annual_premium_groups_low': 0,
        'policy_annual_premium_groups_medium': 1,
        'policy_annual_premium_groups_high': 0,
        'policy_annual_premium_groups_very high': 0
    }
type(temp)

dict

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

preds = svc.predict(X_test)
print('Score:' , svc.score(X_test, y_test))

Score: 0.884


In [25]:
svc_tuned = SVC(kernel='sigmoid', degree = 2, C = 3)
svc_tuned.fit(X_train, y_train)

preds = svc_tuned.predict(X_test)
print('Score:' , svc_tuned.score(X_test, y_test))

Score: 0.884


In [26]:
test = svc_tuned.predict(X_test[[23]])
test

array([1])

In [30]:
# pickle.dump(svc_tuned,open('svc_tuned_fraud_detection.pkl','wb'))